In [1]:
import torch
import numpy as np
import test
import matplotlib.pyplot as plt
from matplotlib import cm
from time import time
import analyse
from importlib import reload
from itertools import permutations
import world
import glob
import plot
from scipy.sparse.csgraph import shortest_path
from copy import deepcopy

In [2]:
def trajectory_len(trajectory):
    return abs(trajectory[1] - trajectory[0]) * 2 + 6

In [3]:
def make_trajectory_dict():
    trajectories = permutations(np.arange(6), r=2)
    trajectory_dict = {}
    for traj_idx, traj in enumerate(trajectories):
        trajectory_dict[traj] = traj_idx
    return trajectory_dict

In [4]:
def locations_in_trajectory(trajectory, environment):
    
    adjacency = np.array(self.adjacency) 
    dists, pred = shortest_path(adjacency, directed=False, method='FW', return_predecessors=True)

    trajectory_locs = [environment.locations[state_id] for state_id in self.get_path(pred, curr_arm_end_state, new_arm_end_state)][1:]
    

In [5]:
def print_proper_time(seconds):
    # hours
    if seconds > 3600:
        output_str = '{0} hours'.format(seconds / 3600)
    # minutes
    elif seconds > 60:
        output_str = '{0} minutes'.format(seconds / 60)
    # seconds
    else:
        output_str = '{0} seconds'.format(seconds)
    return 'Finished running in ' + output_str

In [16]:
IDX = '19000'

In [17]:
# 2022-04-21; run0 uses purely sweeping policy
DATE = '2022-08-02'
RUN = '0'
PATH = '/cumulus/cristofer/TEM_data/{0}/run{1}/model/'.format(DATE, RUN)
SAVE_DIR = '/cumulus/cristofer/TEM_figs/{0}/run{1}/'.format(DATE, RUN)

In [18]:
params = torch.load('/cumulus/cristofer/TEM_data/' + DATE + '/run' + RUN + '/model/params_' + IDX + '.pt')

# Make list of all the environments that this model was trained on
env_list = list(glob.iglob('/cumulus/cristofer/TEM_data/' + DATE + '/run' + RUN + '/script/envs/*'))

# Set which environments will include shiny objects
shiny_envs = [False, False, False, False]
# Set the number of walks to execute in parallel (batch size)
n_walks = len(shiny_envs)
# Select environments from the environments included in training
environments = [world.World(graph, randomise_observations=params['randomise_observations'], shiny=(params['shiny'] if shiny_envs[env_i] else None), specify_behavior=params['specify_behavior'], behavior_type=params['behavior_type'], seed=0) for env_i, graph in enumerate(np.random.choice(env_list, n_walks))]


In [19]:
fwd, mod, envs = test.get_forward(date=DATE, run=RUN, env_to_plot=0, index=IDX, seed=0)

In [20]:
all_g_sweep, all_p_sweep = analyse.trajectories_rate_maps(fwd, mod, envs)